In [1]:
import pandas as pd
import plotly.express as px

# Load data
leads_df = pd.read_csv("leads.csv", parse_dates=["LEAD_CREATED_DATE"])
sales_df = pd.read_csv("sales_funnel.csv", parse_dates=["CASE_OPENED_DATE", "CASE_CLOSED_SUCCESSFUL_DATE"])

# Merge leads with sales funnel
df = sales_df.merge(leads_df, on="LEAD_ID", how="left")

# Create conversion flags
steps = ["Sales Call 1", "Sales Call 2", "PV System Sold"]
for step in steps:
    df[f"{step.lower().replace(' ', '_')}_flag"] = (df["SALES_FUNNEL_STEPS"] == step).astype(int)

# Extract cohort and step months
df["cohort_month"] = df["LEAD_CREATED_DATE"].dt.to_period("M")
df["step_month"] = df["CASE_OPENED_DATE"].dt.to_period("M")

# Calculate time to conversion
df["days_to_convert"] = (df["CASE_CLOSED_SUCCESSFUL_DATE"] - df["LEAD_CREATED_DATE"]).dt.days

# Type conversion
df["cohort_month"] = df["cohort_month"].astype(str)
df["step_month"] = df["step_month"].astype(str)

## lower case for columns
df.columns = df.columns.str.lower()
sales_df.columns = sales_df.columns.str.lower()
leads_df.columns = leads_df.columns.str.lower()


df.to_csv('cohort_analysis_dataset.csv', index= False)
# Preview
df.head()


,lead_id,case_opened_date,case_closed_successful_date,sales_funnel_steps,lead_created_date,marketing_channel,sales_call_1_flag,sales_call_2_flag,pv_system_sold_flag,cohort_month,step_month,days_to_convert
0,cd19e33bf006bfa,2024-06-08,NaT,Sales Call 1,2024-05-21,Channel C,1,0,0,2024-05,2024-06,NaN
1,f4393c6d7ee6891,2024-03-22,2024-03-25,Sales Call 1,2024-02-19,Channel C,1,0,0,2024-02,2024-03,35.0
2,f28e7b7c5295bbc,2024-04-22,2024-04-23,Sales Call 1,2024-03-25,Channel B,1,0,0,2024-03,2024-04,29.0
3,9bae2496eab117a,2024-03-10,2024-03-10,Sales Call 2,2024-02-05,Channel A,0,1,0,2024-02,2024-03,34.0
4,abb172d2a478338,2024-03-21,2024-03-24,Sales Call 1,2024-03-04,Channel C,1,0,0,2024-03,2024-03,20.0


In [2]:

# Define consistent colors for channels
channel_colors = {
    "Channel A": "#1f77b4",  # blue
    "Channel B": "#ff7f0e",  # orange
    "Channel C": "#2ca02c",  # green
}

# Drop duplicate leads (each lead counted once)
leads_per_month_channel = df.drop_duplicates(subset=["lead_id"])

# Plot with color mapping
fig = px.histogram(
    leads_per_month_channel,
    x="cohort_month",
    color="marketing_channel",
    barmode="group",
    title="Monthly Lead Volume by Channel (Cohort View)",
    labels={"cohort_month": "Cohort Month", "count": "Number of Leads"},
    category_orders={"cohort_month": sorted(df["cohort_month"].unique())},
    color_discrete_map=channel_colors
)
fig.show()



In [3]:
import pandas as pd
import plotly.graph_objects as go

# Define consistent colors for channels
channel_colors = {
    "Channel A": "#1f77b4",  # blue
    "Channel B": "#ff7f0e",  # orange
    "Channel C": "#2ca02c",  # green
}

# Total leads
total_leads = df.drop_duplicates(subset=["lead_id"])[["lead_id", "marketing_channel"]]
total_counts = total_leads.groupby("marketing_channel")["lead_id"].count()

# Sales Call 1 entries
call1 = df[df["sales_funnel_steps"] == "Sales Call 1"]
reached_call1 = call1.drop_duplicates(subset=["lead_id"])[["lead_id", "marketing_channel"]]
reached_counts = reached_call1.groupby("marketing_channel")["lead_id"].count()

# Sales Call 1 completions
completed_call1 = call1[call1["case_closed_successful_date"].notna()]
completed_counts = completed_call1.drop_duplicates(subset=["lead_id"]).groupby("marketing_channel")["lead_id"].count()

# Build DataFrame
conversion_df = pd.DataFrame({
    "total_leads": total_counts,
    "reached_call_1": reached_counts,
    "completed_call_1": completed_counts
}).fillna(0)

conversion_df["completion_rate_%"] = round(conversion_df["completed_call_1"] / conversion_df["total_leads"] * 100, 2)
conversion_df["entry_rate_%"] = 100.0  # All leads reached Sales Call 1
conversion_df = conversion_df.reset_index()

# Bar + Line combo plot (color matched)
fig = go.Figure()

# Bar chart for Entry (one bar per channel with mapped color)
for i, row in conversion_df.iterrows():
    fig.add_trace(go.Bar(
        x=[row["marketing_channel"]],
        y=[row["entry_rate_%"]],
        name=f"Entered - {row['marketing_channel']}",
        marker_color=channel_colors.get(row["marketing_channel"], "#999999"),
        showlegend=(i == 0)
    ))

# Line chart for Completion Rate (with matching colors)
# Line chart for Completion Rate (with white percentage labels)
fig.add_trace(go.Scatter(
    x=conversion_df["marketing_channel"],
    y=conversion_df["completion_rate_%"],
    name="Completed Sales Call 1",
    mode="lines+markers+text",
    text=conversion_df["completion_rate_%"].astype(str) + "%",
    textposition="top center",
    textfont=dict(color="white", size=14),
    line=dict(color="grey", width=3)
))


# Layout settings
fig.update_layout(
    title="Sales Call 1 Entry vs Completion Rate by Marketing Channel",
    xaxis_title="Marketing Channel",
    yaxis_title="Rate (%)",
    yaxis_range=[0, 110],
    barmode="group",
    height=500
)

fig.show()


In [4]:

from plotly.subplots import make_subplots



# Prepare data
total_leads = df.drop_duplicates(subset=["lead_id"]).groupby("marketing_channel")["lead_id"].count()
pv_sales = df[df["sales_funnel_steps"] == "PV System Sold"].drop_duplicates(subset=["lead_id"])
pv_sales_counts = pv_sales.groupby("marketing_channel")["lead_id"].count()

# Combine and align channels
channels = sorted(set(total_leads.index).union(set(pv_sales_counts.index)))
conversion_final = pd.DataFrame({
    "marketing_channel": channels,
    "total_leads": [total_leads.get(c, 0) for c in channels],
    "pv_system_sales": [pv_sales_counts.get(c, 0) for c in channels]
})
conversion_final["pv_conversion_rate_%"] = round(
    conversion_final["pv_system_sales"] / conversion_final["total_leads"] * 100, 2
)

# Color mapping
channel_colors = {
    "Channel A": "#1f77b4",  # blue
    "Channel B": "#ff7f0e",  # orange
    "Channel C": "#2ca02c",  # green
}
conversion_final["color"] = conversion_final["marketing_channel"].map(channel_colors)

# Build subplot layout
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.6, 0.4],
    subplot_titles=("PV Conversion Rate by Channel", "Lead Volume Distribution"),
    specs=[[{"type": "bar"}, {"type": "pie"}]]
)

# Bar chart
fig.add_trace(go.Bar(
    x=conversion_final["marketing_channel"],
    y=conversion_final["pv_conversion_rate_%"],
    name="Conversion Rate",
    marker_color=conversion_final["color"],
    text=conversion_final["pv_conversion_rate_%"].astype(str) + "%",
    textposition="outside"
), row=1, col=1)

# Pie chart
fig.add_trace(go.Pie(
    labels=conversion_final["marketing_channel"],
    values=conversion_final["total_leads"],
    name="Total Leads",
    textinfo="label+percent",
    marker=dict(colors=[channel_colors.get(c, "#ccc") for c in conversion_final["marketing_channel"]]),
    hole=0.4
), row=1, col=2)

# Final layout
fig.update_layout(
    title_text="PV System Conversion Rate vs Lead Volume by Channel (Color Matched)",
    showlegend=False,
    height=500
)

fig.show()


In [5]:
import pandas as pd
import plotly.express as px

# STEP 1: Extract Sales Call 1 / 2 leads with their step month
call1_leads = df[df["sales_funnel_steps"] == "Sales Call 1"][["lead_id", "step_month"]].rename(columns={"step_month": "call1_month"})
call2_leads = df[df["sales_funnel_steps"] == "Sales Call 2"][["lead_id", "step_month"]].rename(columns={"step_month": "call2_month"})

# STEP 2: Get all leads who eventually converted (PV system sold)
pv_leads = df[df["sales_funnel_steps"] == "PV System Sold"][["lead_id"]].drop_duplicates()
pv_leads["converted"] = 1

# STEP 3: Merge conversions
call1_leads = call1_leads.drop_duplicates().merge(pv_leads, on="lead_id", how="left").fillna(0)
call2_leads = call2_leads.drop_duplicates().merge(pv_leads, on="lead_id", how="left").fillna(0)

# STEP 4: Group by month and compute conversion %
call1_conv = call1_leads.groupby("call1_month")["converted"].agg(["count", "sum"]).reset_index()
call1_conv["conversion_rate"] = round(call1_conv["sum"] / call1_conv["count"] * 100, 2)
call1_conv["step"] = "Sales Call 1"
call1_conv.rename(columns={"call1_month": "month"}, inplace=True)

call2_conv = call2_leads.groupby("call2_month")["converted"].agg(["count", "sum"]).reset_index()
call2_conv["conversion_rate"] = round(call2_conv["sum"] / call2_conv["count"] * 100, 2)
call2_conv["step"] = "Sales Call 2"
call2_conv.rename(columns={"call2_month": "month"}, inplace=True)

# STEP 5: Combine and plot
combined_conv = pd.concat([call1_conv, call2_conv], ignore_index=True)

fig = px.line(
    combined_conv,
    x="month",
    y="conversion_rate",
    color="step",
    markers=True,
    text="conversion_rate",
    title="📈 Eventual PV Conversion Rate by Sales Call Month",
    labels={"conversion_rate": "Conversion Rate (%)", "month": "Sales Call Month"}
)

fig.update_traces(textposition="top center")
fig.update_layout(yaxis_range=[0, 100], height=500)
fig.show()


In [6]:
import plotly.express as px

# Filter data to include only successful Sales Calls 1 and 2
df_successful_sales_calls = df[
    ((df['sales_funnel_steps'] == 'Sales Call 1') & df['case_closed_successful_date'].notna()) |
    ((df['sales_funnel_steps'] == 'Sales Call 2') & df['case_closed_successful_date'].notna()) |
    (df['sales_funnel_steps'] == 'PV System Sold')
]

# Re-prepare the grouped data
cohort_step_counts = df_successful_sales_calls.groupby(["step_month", "cohort_month"])["lead_id"].nunique().reset_index(name="lead_count")

# Create Stacked Bar Chart for Lead Movement
fig = px.bar(
    cohort_step_counts,
    x="step_month",
    y="lead_count",
    color="cohort_month",
    title="Lead Movement Through the Funnel (Stacked Bar Chart)",
    labels={
        "step_month": "Funnel Step Entry Month",
        "lead_count": "Number of Leads",
        "cohort_month": "Cohort Month"
    },
    height=500
)

fig.update_layout(barmode="stack")
fig.show()


In [7]:
import pandas as pd
import plotly.graph_objects as go

# Define custom colors for Sales Calls and PV System Sold
stage_colors = {
    "Sales Call 1": "#A9D18E",  # Line color for Sales Call 1
    "Sales Call 2": "#66B032",  # Line color for Sales Call 2
    "PV System Sold": "#2DAB2A"  # Bar color for PV System Sold
}

# Filter data to include only successful Sales Call 1 and Sales Call 2 events
df_successful_sales_calls = df[
    ((df['sales_funnel_steps'] == 'Sales Call 1') & df['case_closed_successful_date'].notna()) |
    ((df['sales_funnel_steps'] == 'Sales Call 2') & df['case_closed_successful_date'].notna()) |
    (df['sales_funnel_steps'] == 'PV System Sold')
]

# Prepare the data (grouping by cohort_month and sales funnel steps)
step_distribution = df_successful_sales_calls.groupby(["cohort_month", "sales_funnel_steps"])["lead_id"].nunique().reset_index(name="lead_count")

# Prepare a figure for both lines (Sales Calls) and bar (PV System Sold)
fig = go.Figure()

# Add lines for Sales Call 1 and Sales Call 2
for step in ["Sales Call 1", "Sales Call 2"]:
    filtered_data = step_distribution[step_distribution["sales_funnel_steps"] == step]
    fig.add_trace(go.Scatter(
        x=filtered_data["cohort_month"],
        y=filtered_data["lead_count"],
        mode="lines+markers",
        name=step,
        line=dict(color=stage_colors[step], width=3),
        marker=dict(symbol="circle", size=6)
    ))

# Add bar for PV System Sold
pv_sales_data = step_distribution[step_distribution["sales_funnel_steps"] == "PV System Sold"]
fig.add_trace(go.Bar(
    x=pv_sales_data["cohort_month"],
    y=pv_sales_data["lead_count"],
    name="PV System Sold",
    marker=dict(color=stage_colors["PV System Sold"]),
    text=pv_sales_data["lead_count"],
    textposition="outside",
))

# Update layout for better clarity
fig.update_layout(
    title="📊 Stage-wise Funnel Activity by Cohort",
    xaxis_title="Cohort Month",
    yaxis_title="Number of Leads",
    height=500,
    barmode="stack",  # Stack bars for PV System Sold
    xaxis={'categoryorder': 'total descending'},
    showlegend=True
)

fig.show()
df

,lead_id,case_opened_date,case_closed_successful_date,sales_funnel_steps,lead_created_date,marketing_channel,sales_call_1_flag,sales_call_2_flag,pv_system_sold_flag,cohort_month,step_month,days_to_convert
0,cd19e33bf006bfa,2024-06-08,NaT,Sales Call 1,2024-05-21,Channel C,1,0,0,2024-05,2024-06,NaN
1,f4393c6d7ee6891,2024-03-22,2024-03-25,Sales Call 1,2024-02-19,Channel C,1,0,0,2024-02,2024-03,35.0
2,f28e7b7c5295bbc,2024-04-22,2024-04-23,Sales Call 1,2024-03-25,Channel B,1,0,0,2024-03,2024-04,29.0
3,9bae2496eab117a,2024-03-10,2024-03-10,Sales Call 2,2024-02-05,Channel A,0,1,0,2024-02,2024-03,34.0
4,abb172d2a478338,2024-03-21,2024-03-24,Sales Call 1,2024-03-04,Channel C,1,0,0,2024-03,2024-03,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...
70852,20e104c11bd3d40,2024-06-02,2024-06-02,Sales Call 2,2024-05-12,Channel C,0,1,0,2024-05,2024-06,21.0
70853,54bad5d56f88f3d,2024-08-04,2024-08-04,PV System Sold,2024-05-24,Channel C,0,0,1,2024-05,2024-08,72.0
70854,e46ad0ba724bdb2,2024-05-23,NaT,Sales Call 1,2024-05-02,Channel A,1,0,0,2024-05,2024-05,NaN
70855,28481bb79abd294,2024-07-04,2024-07-05,Sales Call 1,2024-06-17,Channel C,1,0,0,2024-06,2024-07,18.0


In [8]:
import pandas as pd
import plotly.graph_objects as go



# Filter the data for successful Sales Calls (Sales Call 1 and 2) and PV System Sold
df_successful_sales_calls = df[
    ((df['sales_funnel_steps'] == 'Sales Call 1') & df['case_closed_successful_date'].notna()) |
    ((df['sales_funnel_steps'] == 'Sales Call 2') & df['case_closed_successful_date'].notna()) |
    (df['sales_funnel_steps'] == 'PV System Sold')
]

# Group by cohort_month and sales_funnel_steps to calculate the number of leads at each stage
step_counts = df_successful_sales_calls.groupby(
    ['cohort_month', 'sales_funnel_steps'])['lead_id'].nunique().reset_index(name='lead_count')

# Pivot the data to have sales funnel steps as columns for easier comparison
step_counts_pivot = step_counts.pivot_table(
    index="cohort_month", columns="sales_funnel_steps", values="lead_count", aggfunc="sum").reset_index()

# Calculate the drop-off percentages
step_counts_pivot['Sales Call 1 to Sales Call 2 Drop-Off (%)'] = (
    (1 - (step_counts_pivot['Sales Call 2'] / step_counts_pivot['Sales Call 1'])) * 100
)

step_counts_pivot['Sales Call 2 to PV System Sold Drop-Off (%)'] = (
    (1 - (step_counts_pivot['PV System Sold'] / step_counts_pivot['Sales Call 2'])) * 100
)

# Prepare the data for the plot
drop_off_data = {
    'Cohort Month': step_counts_pivot['cohort_month'],
    'Sales Call 1 to Sales Call 2 Drop-Off (%)': step_counts_pivot['Sales Call 1 to Sales Call 2 Drop-Off (%)'],
    'Sales Call 2 to PV System Sold Drop-Off (%)': step_counts_pivot['Sales Call 2 to PV System Sold Drop-Off (%)']
}

# Create a plotly bar chart for the drop-off percentages
fig = go.Figure()


# Add bars for Sales Call 1 to Sales Call 2 drop-off (light red)
fig.add_trace(go.Bar(
    x=drop_off_data['Cohort Month'],
    y=drop_off_data['Sales Call 1 to Sales Call 2 Drop-Off (%)'],
    name="Sales Call 1 to Sales Call 2",
    marker=dict(color='#f08080')  # light red
))

# Add bars for Sales Call 2 to PV System Sold drop-off (medium red)
fig.add_trace(go.Bar(
    x=drop_off_data['Cohort Month'],
    y=drop_off_data['Sales Call 2 to PV System Sold Drop-Off (%)'],
    name="Sales Call 2 to PV System Sold",
    marker=dict(color='#800000')  # medium red
))

# Update layout
fig.update_layout(
    title="Drop-Off Percentages by Cohort Month",
    xaxis_title="Cohort Month",
    yaxis_title="Drop-Off Percentage (%)",
    height=500,
    barmode='group',  # Group bars to show side by side
    showlegend=True
)

fig.show()



In [9]:
import pandas as pd
import plotly.graph_objects as go

# Filter for Sales Call 1 entries
sales_call_1_df = df[df['sales_funnel_steps'] == 'Sales Call 1'].copy()

# Convert dates
sales_call_1_df['lead_created_date'] = pd.to_datetime(sales_call_1_df['lead_created_date'])
sales_call_1_df['case_opened_date'] = pd.to_datetime(sales_call_1_df['case_opened_date'])

# Calculate delay between lead created and sales call 1
sales_call_1_df['days_to_sales_call_1'] = (
    sales_call_1_df['case_opened_date'] - sales_call_1_df['lead_created_date']
).dt.days

# Extract month for grouping
sales_call_1_df['month'] = sales_call_1_df['lead_created_date'].dt.to_period('M').astype(str)

# Group by month and channel
monthly_avg = sales_call_1_df.groupby(['marketing_channel', 'month'])['days_to_sales_call_1'].mean().reset_index()

# Channel colors
channel_colors = {
    "Channel A": "#1f77b4",
    "Channel B": "#ff7f0e",
    "Channel C": "#2ca02c"
}

# Create Plotly figure
fig = go.Figure()

for channel, color in channel_colors.items():
    channel_data = monthly_avg[monthly_avg['marketing_channel'] == channel]
    fig.add_trace(go.Scatter(
        x=channel_data['month'],
        y=channel_data['days_to_sales_call_1'],
        mode='lines+markers',
        name=channel,
        line=dict(color=color)
    ))

fig.update_layout(
    title="Average Days from Lead Created to Sales Call 1 by Channel",
    xaxis_title="Month",
    yaxis_title="Days to Sales Call 1",
    legend_title="Marketing Channel",
    template="plotly_white"
)

fig.show()


In [10]:
import pandas as pd
import plotly.graph_objects as go



# Convert to datetime
df['lead_created_date'] = pd.to_datetime(df['lead_created_date'])
df['case_opened_date'] = pd.to_datetime(df['case_opened_date'])
df['case_closed_successful_date'] = pd.to_datetime(df['case_closed_successful_date'])

# Helper: average days from lead_created_date to step open/close
def get_avg_days(step_name, open_or_close):
    step_df = df[df['sales_funnel_steps'] == step_name]
    if open_or_close == 'opened':
        return (step_df['case_opened_date'] - step_df['lead_created_date']).dt.days.mean()
    elif open_or_close == 'closed':
        return (step_df['case_closed_successful_date'] - step_df['lead_created_date']).dt.days.mean()

# Build average timeline
avg_timeline = {
    'Lead Created': 0,
    'Sales Call 1 - Opened': get_avg_days('Sales Call 1', 'opened'),
    'Sales Call 1 - Closed': get_avg_days('Sales Call 1', 'closed'),
    'Sales Call 2 - Opened': get_avg_days('Sales Call 2', 'opened'),
    'Sales Call 2 - Closed': get_avg_days('Sales Call 2', 'closed'),
    'PV System Sold - Opened': get_avg_days('PV System Sold', 'opened'),
    'PV System Sold - Closed': get_avg_days('PV System Sold', 'closed'),
}

# Plot vertical funnel
stages = list(avg_timeline.keys())
avg_days = list(avg_timeline.values())

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=avg_days,
    y=stages,
    mode='lines+markers+text',
    text=[f"{round(day)}d" for day in avg_days],
    textposition="top center",
    line=dict(color='royalblue', width=4),
    marker=dict(size=10)
))

fig.update_layout(
    title="Average Duration Between Funnel Steps (Days from Lead Creation)",
    xaxis_title="Average Days from Lead Created",
    yaxis_title="Funnel Stage",
    yaxis=dict(autorange="reversed"),
    height=600,
    template="plotly_white"
)

fig.show()


In [11]:

import plotly.graph_objects as go


df['lead_created_date'] = pd.to_datetime(df['lead_created_date'])
df['case_opened_date'] = pd.to_datetime(df['case_opened_date'])
df['case_closed_successful_date'] = pd.to_datetime(df['case_closed_successful_date'])

# Define funnel steps
steps = [
    ('Sales Call 1', 'opened', 'Sales Call 1 - Opened'),
    ('Sales Call 1', 'closed', 'Sales Call 1 - Closed'),
    ('Sales Call 2', 'opened', 'Sales Call 2 - Opened'),
    ('Sales Call 2', 'closed', 'Sales Call 2 - Closed'),
    ('PV System Sold', 'opened', 'PV System Sold - Opened'),
    ('PV System Sold', 'closed', 'PV System Sold - Closed'),
]

# Channel colors
channel_colors = {
    "Channel A": "#1f77b4",  # blue
    "Channel B": "#ff7f0e",  # orange
    "Channel C": "#2ca02c",  # green
}

# Create plot
fig = go.Figure()

for channel, color in channel_colors.items():
    channel_df = df[df['marketing_channel'] == channel]
    timeline = {'Lead Created': 0}

    for step, when, label in steps:
        step_df = channel_df[channel_df['sales_funnel_steps'] == step]
        if when == 'opened':
            avg = (step_df['case_opened_date'] - step_df['lead_created_date']).dt.days.mean()
        else:
            avg = (step_df['case_closed_successful_date'] - step_df['lead_created_date']).dt.days.mean()
        timeline[label] = avg

    # Add trace (no text labels)
    stages = list(timeline.keys())
    avg_days = list(timeline.values())

    fig.add_trace(go.Scatter(
        x=avg_days,
        y=stages,
        mode='lines+markers',
        name=channel,
        line=dict(color=color, width=3),
        marker=dict(size=8)
    ))

fig.update_layout(
    title="Average Duration Between Funnel Steps by Marketing Channel",
    xaxis_title="Days from Lead Created",
    yaxis_title="Funnel Stage",
    yaxis=dict(autorange="reversed"),
    height=700,
    template="plotly_white",
    legend_title="Marketing Channel"
)

fig.show()


In [12]:
import plotly.graph_objects as go


df['lead_created_date'] = pd.to_datetime(df['lead_created_date'])
df['case_opened_date'] = pd.to_datetime(df['case_opened_date'])
df['case_closed_successful_date'] = pd.to_datetime(df['case_closed_successful_date'])
df['month'] = df['lead_created_date'].dt.to_period('M').astype(str)

# Define funnel stages
steps = [
    ('Sales Call 1', 'opened', 'Sales Call 1 - Opened'),
    ('Sales Call 1', 'closed', 'Sales Call 1 - Closed'),
    ('Sales Call 2', 'opened', 'Sales Call 2 - Opened'),
    ('Sales Call 2', 'closed', 'Sales Call 2 - Closed'),
    ('PV System Sold', 'opened', 'PV System Sold - Opened'),
    ('PV System Sold', 'closed', 'PV System Sold - Closed'),
]

# Collect monthly average durations
monthly_avg = {}

for month in df['month'].unique():
    month_df = df[df['month'] == month]
    avg_dict = {'Lead Created': 0}
    
    for step, when, label in steps:
        step_df = month_df[month_df['sales_funnel_steps'] == step]
        if when == 'opened':
            avg = (step_df['case_opened_date'] - step_df['lead_created_date']).dt.days.mean()
        else:
            avg = (step_df['case_closed_successful_date'] - step_df['lead_created_date']).dt.days.mean()
        avg_dict[label] = avg
    
    monthly_avg[month] = avg_dict

# Plot each month
fig = go.Figure()

for month, data in sorted(monthly_avg.items()):
    stages = list(data.keys())
    avg_days = list(data.values())
    fig.add_trace(go.Scatter(
        x=avg_days,
        y=stages,
        mode='lines+markers',
        name=month,
        text=[f"{round(v) if pd.notnull(v) else 'N/A'}d" for v in avg_days],
        textposition="top center"
    ))

fig.update_layout(
    title="Monthly Average Duration Between Funnel Steps",
    xaxis_title="Days from Lead Created",
    yaxis_title="Funnel Stage",
    yaxis=dict(autorange="reversed"),
    height=700,
    template="plotly_white",
    legend_title="Lead Created Month"
)

fig.show()



In [13]:
import pandas as pd
import plotly.graph_objects as go
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Load and prepare data
leads = pd.read_csv("leads.csv")
funnel = pd.read_csv("sales_funnel.csv")



# Convert dates
leads['LEAD_CREATED_DATE'] = pd.to_datetime(leads['LEAD_CREATED_DATE'], errors='coerce')
funnel['CASE_CLOSED_SUCCESSFUL_DATE'] = pd.to_datetime(funnel['CASE_CLOSED_SUCCESSFUL_DATE'], errors='coerce')

# Merge
df = pd.merge(funnel, leads, on="LEAD_ID", how="left")

# Filter only funnel stages of interest
steps = ['Sales Call 1', 'Sales Call 2', 'PV System Sold']
df = df[df['SALES_FUNNEL_STEPS'].isin(steps)]

# Compute duration
df['days_to_stage_closed'] = (df['CASE_CLOSED_SUCCESSFUL_DATE'] - df['LEAD_CREATED_DATE']).dt.days

# Setup plot
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=[
        "Lead → Sales Call 1 Closed",
        "Lead → Sales Call 2 Closed",
        "Lead → PV System Closed"
    ]
)

colors = ['lightblue', 'lightgreen', 'salmon']
short_labels = ['Call 1 Closed', 'Call 2 Closed', 'PV Closed']

# Add plots and lines
for i, (step, color, label) in enumerate(zip(steps, colors, short_labels)):
    step_data = df[df['SALES_FUNNEL_STEPS'] == step]['days_to_stage_closed'].dropna()
    p75 = step_data.quantile(0.75)
    p90 = step_data.quantile(0.90)

    # Histogram
    fig.add_trace(go.Histogram(
        x=step_data,
        nbinsx=30,
        marker_color=color,
        name=label,
        showlegend=True,
        legendgroup=label
    ), row=1, col=i+1)

    # 75th percentile line
    fig.add_shape(type="line",
        x0=p75, x1=p75, y0=0, y1=1,
        xref=f"x{i+1}", yref="paper",
        line=dict(color="blue", dash="dot")
    )

    fig.add_annotation(
        x=p75, y=1.02,
        text=f"75%: {int(p75)}d",
        showarrow=False,
        xref=f"x{i+1}", yref="paper",
        font=dict(color="blue")
    )

    # 90th percentile as bottom text only (no line)
    fig.add_annotation(
        x=p90, y=0,
        text=f"90%: {int(p90)}d",
        showarrow=False,
        xref=f"x{i+1}", yref="paper",
        font=dict(color="black", size=12),
        xanchor="left",
        yanchor="bottom"
    )

# Final layout
fig.update_layout(
    title="Lead Inactivity Safe Window",
    height=500,
    width=1300,
    bargap=0.1,
    legend_title="Safe Window Percentile"
)

fig.update_xaxes(title_text="Days")
fig.update_yaxes(title_text="Lead Count")

fig.show()
